### Install Dependencies

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install datasets scipy ipywidgets matplotlib torch accelerate
!pip install wandb -qU
!pip install --upgrade transformers huggingface_hub
!pip install evaluate rouge_score

### Initialize Wanb

In [ ]:
import wandb
from google.colab import userdata
wandb.login(key=userdata.get('WANDB_TOKEN')) #Your WandB acces token here
wandb.init(
    project="PerspectrumInstruct-original-NoFineTuning-Unsloth_Llamma8B",
    name='Text ouput capture',
    tags=["vanilla", "of the shelf", "No-fine-tuning"],
    notes="Original off the shelf model with no fine-tuning to check if the model can work or not"
    )

In [ ]:
table = wandb.Table(columns=["instruction", "claim", "response"])

### Load model and tokenizer

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype=None
load_in_4bit=True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", #
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)# Enable native 2x faster inference

### Format Input prompts

In [ ]:
System_prompt = "Below is an instruction that describes an information requirement, paired with a claim that provides context. Write a response that appropriately addresses the instruction based on the given claim."
Input_prompt = "### Instruction:\n{instruction}\n\n### Claim:\n{claim}\n\n### Response:\n{answer}"
text = Input_prompt.format(instruction='What is the distribution of opinions about the following claim?',
                           claim='Religious symbols in public buildings must be banned.',
                           answer='',)
full_prompt = f"{System_prompt}\n\n{text}"
print(full_prompt)

inputs = tokenizer(
[
    full_prompt,
], return_tensors = "pt").to(torch.device("cuda"))

Below is an instruction that describes an information requirement, paired with a claim that provides context. Write a response that appropriately addresses the instruction based on the given claim.

### Instruction:
What is the distribution of opinions about the following claim?

### Claim:
Religious symbols in public buildings must be banned.

### Response:



In [ ]:
inputs

{'input_ids': tensor([[128000,  39314,    374,    459,   7754,    430,  16964,    459,   2038,
          16686,     11,  35526,    449,    264,   3802,    430,   5825,   2317,
             13,   9842,    264,   2077,    430,  36001,  14564,    279,   7754,
           3196,    389,    279,   2728,   3802,    382,  14711,  30151,    512,
           3923,    374,    279,   8141,    315,  18463,    922,    279,   2768,
           3802,   1980,  14711,  38602,    512,   6882,  22941,  18210,    304,
            586,  14016,   2011,    387,  21501,    382,  14711,   6075,    512]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

### Generate Model output

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [ ]:
output = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes an information requirement, paired with a claim that provides context. Write a response that appropriately addresses the instruction based on the given claim.

### Instruction:
What is the distribution of opinions about the following claim?

### Claim:
Religious symbols in public buildings must be banned.

### Response:
A poll of 1000 people found that 35% of people agreed with the claim, 37% of people disagreed with the claim, and 28% of people were unsure about the claim.<|end_of_text|>


In [ ]:
gen_text = tokenizer.decode(output[0])

In [ ]:
gen_text

'<|begin_of_text|>Below is an instruction that describes an information requirement, paired with a claim that provides context. Write a response that appropriately addresses the instruction based on the given claim.\n\n### Instruction:\nWhat is the distribution of opinions about the following claim?\n\n### Claim:\nReligious symbols in public buildings must be banned.\n\n### Response:\nA poll of 1000 people found that 35% of people agreed with the claim, 37% of people disagreed with the claim, and 28% of people were unsure about the claim.<|end_of_text|>'

In [ ]:
table.add_data('What is the distribution of opinions about the following claim?', 'Religious symbols in public buildings must be banned.', gen_text)
wandb.log({"Generations":table})

In [ ]:
wandb.finish()